## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
# week_ending = format_date_with_ordinal(today)
# week_starting = format_date_with_ordinal(one_week_ago)

week_ending = 'Dec. 9th'
week_starting = 'Dec. 2nd'

print(week_starting, week_ending)

Dec. 2nd Dec. 9th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
len(df)

686

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:

# Define the date range
start_date = '2024-12-02'
end_date = '2024-12-09'

# Filter the DataFrame between two dates (inclusive)
df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [11]:
print('After date restriction: ',len(df))

After date restriction:  154


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [12]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
111,PAST SALE,2024-12-09,Condo/Co-op,10456 NW 24th Pl #301,Sunrise,FL,33322.0,150000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunrise/10456-NW-24t...,MARMLS,A11635633,N,Y,26.156337,-80.289091
86,PAST SALE,2024-12-09,Condo/Co-op,815 Middle River Dr #317,Fort Lauderdale,FL,33304.0,285000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/815-...,MARMLS,A11663579,N,Y,26.135641,-80.118213
205,PAST SALE,2024-12-09,Condo/Co-op,9599 Weldon Cir #208,Tamarac,FL,33321.0,205000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Tamarac/9599-Weldon-...,Beaches MLS,F10452712,N,Y,26.221338,-80.280161
66,PAST SALE,2024-12-09,Condo/Co-op,12750 SW 15th St Unit 308D,Pembroke Pines,FL,33027.0,179900.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pembroke-Pines/12750...,MARMLS,A11682577,N,Y,25.994780,-80.317040
207,PAST SALE,2024-12-09,Condo/Co-op,4140 NW 44th Ave #305,Lauderdale Lakes,FL,33319.0,105000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-Lakes/414...,MARMLS,A11571852,N,Y,26.176631,-80.207285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,PAST SALE,2024-12-02,Condo/Co-op,3300 Spanish Moss Ter #411,Lauderhill,FL,33319.0,70000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderhill/3300-Span...,Beaches MLS,F10455001,N,Y,26.167150,-80.232701
174,PAST SALE,2024-12-02,Condo/Co-op,2676 SW 120th Ter Unit 805-8,Miramar,FL,33025.0,400000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miramar/2676-SW-120t...,MARMLS,A11646036,N,Y,25.982815,-80.306722
123,PAST SALE,2024-12-02,Condo/Co-op,103 Royal Park Dr Unit 1G,Oakland Park,FL,33309.0,160000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Oakland-Park/103-Roy...,Beaches MLS,F10454038,N,Y,26.171827,-80.163722
189,PAST SALE,2024-12-02,Condo/Co-op,4500 N Federal Hwy Unit 119-C,Lighthouse Point,FL,33064.0,210000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lighthouse-Point/450...,Beaches MLS,RX-11012486,N,Y,26.286911,-80.094864


In [13]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [14]:
df_top_ten = df.head(10)

## Collect Agent Information

In [15]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [16]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [17]:
response_df = pd.DataFrame(response_list)

In [18]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [19]:
df_top_ten = merged_df

## Current Week's Values

In [20]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

154


In [21]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$379,013


In [22]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$287


In [23]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$58,368,044


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [27]:
# Define the date range
# start_date = '2024-09-08'
# end_date = '2024-09-15'

# Filter the DataFrame between two dates (inclusive)
# filtered_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [24]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Broward
Input Previous Week Condo Sales Total (number units sold):
119
Input Previous Week Condo Average Sales Price:
288204
Input Previous Week Condo Average PSF:
261
Input Previous Week Condo Sales total (ex: 198_000_000)
34_300_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [25]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [26]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [27]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [28]:
pd.set_option('display.max_columns',None)

In [29]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-12-05,Condo/Co-op,3100 N Ocean Blvd #2810,Fort Lauderdale,FL,33308.0,5650000.0,4.0,5.5,Lhermitage I Condominium,4630.0,NaN,1997.0,NaN,1220.0,5660.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Beaches MLS,RX-10990250,N,Y,26.168258,-80.099279,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Deborah Ginsburg,"Nest Seekers Florida, LLC",None,None,Victor Attias,Galt Ocean Realty Inc,None,None,1,orange


In [30]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [31]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-12-05,Condo/Co-op,3100 N Ocean Blvd #2810,Fort Lauderdale,FL,33308.0,5650000.0,4.0,5.5,Lhermitage I Condominium,4630.0,NaN,1997.0,NaN,1220.0,5660.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Beaches MLS,RX-10990250,N,Y,26.168258,-80.099279,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Deborah Ginsburg,"Nest Seekers Florida, LLC",None,None,Victor Attias,Galt Ocean Realty Inc,None,None,1,orange
1,PAST SALE,2024-12-02,Condo/Co-op,141 W Isle Of Venice Dr Ph -S,Fort Lauderdale,FL,33301.0,3775000.0,3.0,4.5,Casa Murano,3131.0,NaN,2024.0,NaN,1206.0,2521.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/141-...,Beaches MLS,F10450114,N,Y,26.125672,-80.120892,https://www.redfin.com/FL/Fort-Lauderdale/141-...,Ruchel Coetzee,Douglas Elliman,None,None,Tyler Tuchow,Fortune Christie's International Real Estate,None,None,2,blue
2,PAST SALE,2024-12-06,Condo/Co-op,1116 N Ocean Blvd #601,Pompano Beach,FL,33062.0,3100000.0,3.0,3.5,Solemar,2593.0,NaN,2023.0,NaN,1196.0,3480.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1116-N...,Beaches MLS,F10429014,N,Y,26.247536,-80.086777,https://www.redfin.com/FL/Pompano-Beach/1116-N...,Anthony Albino,Beautiful Homes Realty Inc,None,None,Non-member Mls Agent,NON MEMBER MLS,None,None,3,blue
3,PAST SALE,2024-12-03,Condo/Co-op,3100 N Ocean Blvd #908,Fort Lauderdale,FL,33308.0,2500000.0,3.0,3.5,L'hermitage,2318.0,NaN,1997.0,NaN,1079.0,3119.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Beaches MLS,F10454106,N,Y,26.168258,-80.099279,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Vicci Read,Florida Luxurious Properties,None,None,James Cunningham,One Sotheby's Int'l Realty,None,None,4,blue
4,PAST SALE,2024-12-09,Condo/Co-op,411 N New River Dr #1406,Fort Lauderdale,FL,33301.0,1650000.0,3.0,3.5,Las Olas Grand,2635.0,NaN,2005.0,NaN,626.0,2884.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Beaches MLS,F10428680,N,Y,26.118085,-80.138584,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Dennis Miller,Better Homes & Gdns RE Fla 1st,None,None,Sharon O'Connor,Coldwell Banker Realty,None,None,5,blue
5,PAST SALE,2024-12-06,Condo/Co-op,6051 N Ocean Dr #706,Hollywood,FL,33019.0,1450000.0,2.0,2.5,RENAISSANCE ON THE OCEAN,1992.0,NaN,1999.0,NaN,728.0,2502.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/6051-N-Oce...,MARMLS,A11654583,N,Y,26.052035,-80.113046,https://www.redfin.com/FL/Hollywood/6051-N-Oce...,Gustavo Somoano,GSL Properties,None,None,None,None,None,None,6,blue
6,PAST SALE,2024-12-04,Condo/Co-op,1830 S Ocean Dr #5008,Hallandale Beach,FL,33009.0,1267000.0,2.0,2.0,BEACH CLUB TWO CONDO,1458.0,NaN,2006.0,NaN,869.0,1300.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/183...,MARMLS,A11674437,N,Y,25.984911,-80.118344,https://www.redfin.com/FL/Hallandale-Beach/183...,Svetlana Perez,"LP Global Realty, LLC",Mariya Vlasova,"LP Global Realty, LLC",Gloria Chenlo,Cervera Real Estate Inc,None,None,7,blue
7,PAST SALE,2024-12-03,Condo/Co-op,1201 S Ocean Dr Unit 1801S,Hollywood,FL,33019.0,925000.0,3.0,2.5,SUMMIT CONDO,1990.0,NaN,1982.0,NaN,465.0,1861.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/1201-S-Oce...,MARMLS,A11658219,N,Y,26.005693,-80.117761,https://www.redfin.com/FL/Hollywood/1201-S-Oce...,Amy Greenberg,Summit Real Estate Brokerage,Melissa Cooperman,Summit Real Estate Brokerage,Amy Greenberg,Summit Real Estate Brokerage,None,None,8,blue
8,PAST SALE,2024-12-03,Condo/Co-op,2100 S Ocean Ln #1103,Fort Lauderdale,FL,33316.0,910000.0,2.0,2.0,Point Of Americas,1500.0,NaN,1970.0,N

In [33]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Broward County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [34]:
m.save('index.html')

## Data snagger

In [35]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [36]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-12-05,Condo/Co-op,3100 N Ocean Blvd #2810,Fort Lauderdale,FL,33308.0,5650000.0,4.0,5.5,Lhermitage I Condominium,4630.0,NaN,1997.0,NaN,1220.0,5660.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Beaches MLS,RX-10990250,N,Y,26.168258,-80.099279,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Deborah Ginsburg,"Nest Seekers Florida, LLC",None,None,Victor Attias,Galt Ocean Realty Inc,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [37]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Broward_County_condo_sales_week_ending_12092024


# CREATE TEMPLATE 

In [38]:
muni_set = set(df_top_ten['CITY'])

In [39]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [40]:
df_top_ten.reset_index(inplace=True,drop=True)

In [41]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [42]:
top_sale

'Lhermitage I Condominium at 3100 N Ocean Blvd #2810 in Fort Lauderdale'

In [43]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [44]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [45]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-12-05,Condo/Co-op,3100 N Ocean Blvd #2810,Fort Lauderdale,FL,33308.0,5650000.0,4.0,5.5,Lhermitage I Condominium,4630.0,NaN,1997.0,NaN,1220.0,5660.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Beaches MLS,RX-10990250,N,Y,26.168258,-80.099279,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Deborah Ginsburg,"Nest Seekers Florida, LLC",None,None,Victor Attias,Galt Ocean Realty Inc,None,None,1,orange,Lhermitage I Condominium at 3100 N Ocean Blvd ...
1,PAST SALE,2024-12-02,Condo/Co-op,141 W Isle Of Venice Dr Ph -S,Fort Lauderdale,FL,33301.0,3775000.0,3.0,4.5,Casa Murano,3131.0,NaN,2024.0,NaN,1206.0,2521.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/141-...,Beaches MLS,F10450114,N,Y,26.125672,-80.120892,https://www.redfin.com/FL/Fort-Lauderdale/141-...,Ruchel Coetzee,Douglas Elliman,None,None,Tyler Tuchow,Fortune Christie's International Real Estate,None,None,2,blue,Casa Murano at 141 W Isle Of Venice Dr Ph -S i...
2,PAST SALE,2024-12-06,Condo/Co-op,1116 N Ocean Blvd #601,Pompano Beach,FL,33062.0,3100000.0,3.0,3.5,Solemar,2593.0,NaN,2023.0,NaN,1196.0,3480.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1116-N...,Beaches MLS,F10429014,N,Y,26.247536,-80.086777,https://www.redfin.com/FL/Pompano-Beach/1116-N...,Anthony Albino,Beautiful Homes Realty Inc,None,None,Non-member Mls Agent,NON MEMBER MLS,None,None,3,blue,Solemar at 1116 N Ocean Blvd #601 in Pompano B...
3,PAST SALE,2024-12-03,Condo/Co-op,3100 N Ocean Blvd #908,Fort Lauderdale,FL,33308.0,2500000.0,3.0,3.5,L'hermitage,2318.0,NaN,1997.0,NaN,1079.0,3119.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Beaches MLS,F10454106,N,Y,26.168258,-80.099279,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Vicci Read,Florida Luxurious Properties,None,None,James Cunningham,One Sotheby's Int'l Realty,None,None,4,blue,L'hermitage at 3100 N Ocean Blvd #908 in Fort ...
4,PAST SALE,2024-12-09,Condo/Co-op,411 N New River Dr #1406,Fort Lauderdale,FL,33301.0,1650000.0,3.0,3.5,Las Olas Grand,2635.0,NaN,2005.0,NaN,626.0,2884.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Beaches MLS,F10428680,N,Y,26.118085,-80.138584,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Dennis Miller,Better Homes & Gdns RE Fla 1st,None,None,Sharon O'Connor,Coldwell Banker Realty,None,None,5,blue,Las Olas Grand at 411 N New River Dr #1406 in ...
5,PAST SALE,2024-12-06,Condo/Co-op,6051 N Ocean Dr #706,Hollywood,FL,33019.0,1450000.0,2.0,2.5,RENAISSANCE ON THE OCEAN,1992.0,NaN,1999.0,NaN,728.0,2502.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/6051-N-Oce...,MARMLS,A11654583,N,Y,26.052035,-80.113046,https://www.redfin.com/FL/Hollywood/6051-N-Oce...,Gustavo Somoano,GSL Properties,None,None,None,None,None,None,6,blue,RENAISSANCE ON THE OCEAN at 6051 N Ocean Dr #7...
6,PAST SALE,2024-12-04,Condo/Co-op,1830 S Ocean Dr #5008,Hallandale Beach,FL,33009.0,1267000.0,2.0,2.0,BEACH CLUB TWO CONDO,1458.0,NaN,2006.0,NaN,869.0,1300.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/183...,MARMLS,A11674437,N,Y,25.984911,-80.118344,https://www.redfin.com/FL/Hallandale-Beach/183...,Svetlana Perez,"LP Global Realty, LLC",Mariya Vlasova,"LP Global Realty, LLC",Gloria Chenlo,Cervera Real Estate Inc,None,None,7,blue,BEACH CLUB TWO CONDO at 1830 S Ocean Dr #5008 ...
7,PAST SALE,2024-12-03,Condo/Co-op,1201 S Ocean Dr Unit 1801S,Hollywood,FL,33019.0,925000.0,3.0,2.5,SUMMIT CONDO,1990.0,NaN,1982.0,NaN,465.0,1861.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/1201-S-Oce...,MARMLS,A11658219,N,Y

In [46]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Lhermitage I Condominium closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $700,000M to $5,650,000M
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Broward condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Hollywood, Hallandale Beach, Pompano Beach, Fort Lauderdale

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 154 condo sales totaling $58,368,044 million from Dec. 2nd to Dec. 9th. The previous week, brokers closed 119 condo sales totaling $34,300,000.

Last week’s units sold for an average of $379,013, higher than the $288,204 average price from the previous week. The average p

In [47]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [48]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [49]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [50]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-12-05,Condo/Co-op,3100 N Ocean Blvd #2810,Fort Lauderdale,FL,33308.0,5650000.0,4.0,5.5,Lhermitage I Condominium,4630.0,NaN,1997.0,NaN,1220.0,5660.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Beaches MLS,RX-10990250,N,Y,26.168258,-80.099279,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Deborah Ginsburg,"Nest Seekers Florida, LLC",None,None,Victor Attias,Galt Ocean Realty Inc,None,None,1,orange,Lhermitage I Condominium at 3100 N Ocean Blvd ...
1,PAST SALE,2024-12-02,Condo/Co-op,141 W Isle Of Venice Dr Ph -S,Fort Lauderdale,FL,33301.0,3775000.0,3.0,4.5,Casa Murano,3131.0,NaN,2024.0,NaN,1206.0,2521.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/141-...,Beaches MLS,F10450114,N,Y,26.125672,-80.120892,https://www.redfin.com/FL/Fort-Lauderdale/141-...,Ruchel Coetzee,Douglas Elliman,None,None,Tyler Tuchow,Fortune Christie's International Real Estate,None,None,2,blue,Casa Murano at 141 W Isle Of Venice Dr Ph -S i...
2,PAST SALE,2024-12-06,Condo/Co-op,1116 N Ocean Blvd #601,Pompano Beach,FL,33062.0,3100000.0,3.0,3.5,Solemar,2593.0,NaN,2023.0,NaN,1196.0,3480.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1116-N...,Beaches MLS,F10429014,N,Y,26.247536,-80.086777,https://www.redfin.com/FL/Pompano-Beach/1116-N...,Anthony Albino,Beautiful Homes Realty Inc,None,None,Non-member Mls Agent,NON MEMBER MLS,None,None,3,blue,Solemar at 1116 N Ocean Blvd #601 in Pompano B...
3,PAST SALE,2024-12-03,Condo/Co-op,3100 N Ocean Blvd #908,Fort Lauderdale,FL,33308.0,2500000.0,3.0,3.5,L'hermitage,2318.0,NaN,1997.0,NaN,1079.0,3119.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Beaches MLS,F10454106,N,Y,26.168258,-80.099279,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Vicci Read,Florida Luxurious Properties,None,None,James Cunningham,One Sotheby's Int'l Realty,None,None,4,blue,L'hermitage at 3100 N Ocean Blvd #908 in Fort ...
4,PAST SALE,2024-12-09,Condo/Co-op,411 N New River Dr #1406,Fort Lauderdale,FL,33301.0,1650000.0,3.0,3.5,Las Olas Grand,2635.0,NaN,2005.0,NaN,626.0,2884.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Beaches MLS,F10428680,N,Y,26.118085,-80.138584,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Dennis Miller,Better Homes & Gdns RE Fla 1st,None,None,Sharon O'Connor,Coldwell Banker Realty,None,None,5,blue,Las Olas Grand at 411 N New River Dr #1406 in ...
5,PAST SALE,2024-12-06,Condo/Co-op,6051 N Ocean Dr #706,Hollywood,FL,33019.0,1450000.0,2.0,2.5,RENAISSANCE ON THE OCEAN,1992.0,NaN,1999.0,NaN,728.0,2502.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/6051-N-Oce...,MARMLS,A11654583,N,Y,26.052035,-80.113046,https://www.redfin.com/FL/Hollywood/6051-N-Oce...,Gustavo Somoano,GSL Properties,None,None,None,None,None,None,6,blue,RENAISSANCE ON THE OCEAN at 6051 N Ocean Dr #7...
6,PAST SALE,2024-12-04,Condo/Co-op,1830 S Ocean Dr #5008,Hallandale Beach,FL,33009.0,1267000.0,2.0,2.0,BEACH CLUB TWO CONDO,1458.0,NaN,2006.0,NaN,869.0,1300.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/183...,MARMLS,A11674437,N,Y,25.984911,-80.118344,https://www.redfin.com/FL/Hallandale-Beach/183...,Svetlana Perez,"LP Global Realty, LLC",Mariya Vlasova,"LP Global Realty, LLC",Gloria Chenlo,Cervera Real Estate Inc,None,None,7,blue,BEACH CLUB TWO CONDO at 1830 S Ocean Dr #5008 ...
7,PAST SALE,2024-12-03,Condo/Co-op,1201 S Ocean Dr Unit 1801S,Hollywood,FL,33019.0,925000.0,3.0,2.5,SUMMIT CONDO,1990.0,NaN,1982.0,NaN,465.0,1861.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/1201-S-Oce...,MARMLS,A11658219,N,Y

In [51]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: Lhermitage I Condominium closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $700,000M to $5,650,000M
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Broward condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Hollywood, Hallandale Beach, Pompano Beach, Fort Lauderdale

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 154 condo sales totaling $58,368,044 million from Dec. 2nd to Dec. 9th. The previous week, brokers closed 119 condo sales totaling $34,300,000.

Last week’s units sold for an average of $379,013, higher than the $288,204 average price from the previous week. The average p

In [59]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Fort-Lauderdale/3100-N-Ocean-Blvd-33308/unit-2810/home/41715597


In [60]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Fort-Lauderdale/141-Isle-of-Venice-Dr-33301/unit-S/home/190923029


In [61]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Hallandale-Beach/1880-S-Ocean-Dr-33009/unit-1/home/41963614


In [62]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Fort-Lauderdale/2100-S-Ocean-Ln-33316/unit-1103/home/41805610


In [63]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Fort-Lauderdale/3100-N-Ocean-Blvd-33308/unit-2810/home/41715597


In [64]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Hallandale-Beach/1880-S-Ocean-Dr-33009/unit-1/home/41963614


In [65]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-12-05,Condo/Co-op,3100 N Ocean Blvd #2810,Fort Lauderdale,FL,33308.0,5650000.0,4.0,5.5,Lhermitage I Condominium,4630.0,NaN,1997.0,NaN,1220.0,5660.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Beaches MLS,RX-10990250,N,Y,26.168258,-80.099279,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Deborah Ginsburg,"Nest Seekers Florida, LLC",None,None,Victor Attias,Galt Ocean Realty Inc,None,None,1,orange,Lhermitage I Condominium at 3100 N Ocean Blvd ...
1,PAST SALE,2024-12-02,Condo/Co-op,141 W Isle Of Venice Dr Ph -S,Fort Lauderdale,FL,33301.0,3775000.0,3.0,4.5,Casa Murano,3131.0,NaN,2024.0,NaN,1206.0,2521.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/141-...,Beaches MLS,F10450114,N,Y,26.125672,-80.120892,https://www.redfin.com/FL/Fort-Lauderdale/141-...,Ruchel Coetzee,Douglas Elliman,None,None,Tyler Tuchow,Fortune Christie's International Real Estate,None,None,2,blue,Casa Murano at 141 W Isle Of Venice Dr Ph -S i...
2,PAST SALE,2024-12-06,Condo/Co-op,1116 N Ocean Blvd #601,Pompano Beach,FL,33062.0,3100000.0,3.0,3.5,Solemar,2593.0,NaN,2023.0,NaN,1196.0,3480.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1116-N...,Beaches MLS,F10429014,N,Y,26.247536,-80.086777,https://www.redfin.com/FL/Pompano-Beach/1116-N...,Anthony Albino,Beautiful Homes Realty Inc,None,None,Non-member Mls Agent,NON MEMBER MLS,None,None,3,blue,Solemar at 1116 N Ocean Blvd #601 in Pompano B...
3,PAST SALE,2024-12-03,Condo/Co-op,3100 N Ocean Blvd #908,Fort Lauderdale,FL,33308.0,2500000.0,3.0,3.5,L'hermitage,2318.0,NaN,1997.0,NaN,1079.0,3119.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Beaches MLS,F10454106,N,Y,26.168258,-80.099279,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Vicci Read,Florida Luxurious Properties,None,None,James Cunningham,One Sotheby's Int'l Realty,None,None,4,blue,L'hermitage at 3100 N Ocean Blvd #908 in Fort ...
4,PAST SALE,2024-12-09,Condo/Co-op,411 N New River Dr #1406,Fort Lauderdale,FL,33301.0,1650000.0,3.0,3.5,Las Olas Grand,2635.0,NaN,2005.0,NaN,626.0,2884.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Beaches MLS,F10428680,N,Y,26.118085,-80.138584,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Dennis Miller,Better Homes & Gdns RE Fla 1st,None,None,Sharon O'Connor,Coldwell Banker Realty,None,None,5,blue,Las Olas Grand at 411 N New River Dr #1406 in ...
5,PAST SALE,2024-12-06,Condo/Co-op,6051 N Ocean Dr #706,Hollywood,FL,33019.0,1450000.0,2.0,2.5,RENAISSANCE ON THE OCEAN,1992.0,NaN,1999.0,NaN,728.0,2502.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/6051-N-Oce...,MARMLS,A11654583,N,Y,26.052035,-80.113046,https://www.redfin.com/FL/Hollywood/6051-N-Oce...,Gustavo Somoano,GSL Properties,None,None,None,None,None,None,6,blue,RENAISSANCE ON THE OCEAN at 6051 N Ocean Dr #7...
6,PAST SALE,2024-12-04,Condo/Co-op,1830 S Ocean Dr #5008,Hallandale Beach,FL,33009.0,1267000.0,2.0,2.0,BEACH CLUB TWO CONDO,1458.0,NaN,2006.0,NaN,869.0,1300.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/183...,MARMLS,A11674437,N,Y,25.984911,-80.118344,https://www.redfin.com/FL/Hallandale-Beach/183...,Svetlana Perez,"LP Global Realty, LLC",Mariya Vlasova,"LP Global Realty, LLC",Gloria Chenlo,Cervera Real Estate Inc,None,None,7,blue,BEACH CLUB TWO CONDO at 1830 S Ocean Dr #5008 ...
7,PAST SALE,2024-12-03,Condo/Co-op,1201 S Ocean Dr Unit 1801S,Hollywood,FL,33019.0,925000.0,3.0,2.5,SUMMIT CONDO,1990.0,NaN,1982.0,NaN,465.0,1861.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/1201-S-Oce...,MARMLS,A11658219,N,Y

## Time on Market Calculator

In [67]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 7, 12) ## List (Earlier) date
date2 = datetime(2024, 12, 12) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

153


## Clean CSV for Datawrapper Chart

In [68]:
chart_df = df_top_ten

In [69]:
chart_df = chart_df.fillna(" ")

In [70]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [71]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [72]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [73]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [74]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [75]:
csv_date_string = today.strftime("%m_%d_%Y")

In [76]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [77]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2024-12-05,3100 N Ocean Blvd #2810 in Fort Lauderdale,"$5,650,000",4,5.5,"4,630",1997,"$1,220",Beaches MLS,RX-10990250,"Deborah Ginsburg Nest Seekers Florida, LLC",Victor Attias Galt Ocean Realty Inc
1,2024-12-02,141 W Isle Of Venice Dr Ph -S in Fort Lauderdale,"$3,775,000",3,4.5,"3,131",2024,"$1,206",Beaches MLS,F10450114,Ruchel Coetzee Douglas Elliman,Tyler Tuchow Fortune Christie's International ...
2,2024-12-06,1116 N Ocean Blvd #601 in Pompano Beach,"$3,100,000",3,3.5,"2,593",2023,"$1,196",Beaches MLS,F10429014,Anthony Albino Beautiful Homes Realty Inc,Non-member Mls Agent NON MEMBER MLS
3,2024-12-03,3100 N Ocean Blvd #908 in Fort Lauderdale,"$2,500,000",3,3.5,"2,318",1997,"$1,079",Beaches MLS,F10454106,Vicci Read Florida Luxurious Properties,James Cunningham One Sotheby's Int'l Realty
4,2024-12-09,411 N New River Dr #1406 in Fort Lauderdale,"$1,650,000",3,3.5,"2,635",2005,$626,Beaches MLS,F10428680,Dennis Miller Better Homes & Gdns RE Fla 1st,Sharon O'Connor Coldwell Banker Realty
5,2024-12-06,6051 N Ocean Dr #706 in Hollywood,"$1,450,000",2,2.5,"1,992",1999,$728,MARMLS,A11654583,Gustavo Somoano GSL Properties,
6,2024-12-04,1830 S Ocean Dr #5008 in Hallandale Beach,"$1,267,000",2,2.0,"1,458",2006,$869,MARMLS,A11674437,"Svetlana Perez LP Global Realty, LLC Mariya Vl...",Gloria Chenlo Cervera Real Estate Inc
7,2024-12-03,1201 S Ocean Dr Unit 1801S in Hollywood,"$925,000",3,2.5,"1,990",1982,$465,MARMLS,A11658219,Amy Greenberg Summit Real Estate Brokerage Mel...,Amy Greenberg Summit Real Estate Brokerage
8,2024-12-03,2100 S Ocean Ln #1103 in Fort Lauderdale,"$910,000",2,2.0,"1,500",1970,$607,Beaches MLS,F10447836,Tyler Tuchow Fortune Christie's International ...,Nancy Connor Paradise Real Estate Intl
9,2024-12-02,1880 S Ocean Dr Ph 1 in Hallandale Beach,"$700,000",3,3.0,"1,930",1973,$363,MARMLS,A11674051,Patricia Garcia Equity Real Property and Mgt. ...,"Olga Zaurova, PA Big International Realty, Inc..."
